In [1]:
import pickle
import re
import numpy as np
import pandas as pd
import torch


# from google.colab import drive
# drive.mount('/content/drive')

In [18]:
with open('data/mentions texts.pickle', 'rb') as f:
    news_train = pickle.load(f)


with open('data/sentiment_texts.pickle', 'rb') as f:
    sentiment_train = pickle.load(f)

In [195]:
test_text = news_train.iloc[7].MessageText
test_text

'Главное к открытию среды (20.09): #брифинг  📌 Значение индекса ЖиС: 60 (жадность). Подробнее об индексе здесь.  📍 Курсы валют: 96.7 ₽/$ (-0.1%), 13.2 ₽/¥ (-0.4%).  📍 Мосбиржа (MOEX) c 21 сентября 2023 г. вводит ограничение на максимальное отклонение цен заявок, подаваемых участниками рынка на приобретение акций и депозитарных расписок российских эмитентов, включенных в третий уровень листинга, на уровне не выше 22% от цены закрытия предыдущего торгового дня. 📍 Русснефть (RNFT) сократит добычу в 2023 г. на 5% из-за решений ОПЕК+. Русснефть назвала уровень цен на российскую нефть в $72 за баррель комфортным. 📍 События сегодня: (1) США: решение по процентной ставке 21:00 мск и пресс-конференция Пауэлла 21:30 мск. (2) Запасы сырой нефти в США 17:30 мск. (3) Positive Technologies (POSI): четвертый заключительный раунд выкупа в рамках SPO в акциях.   📈Что влияет на рынки в ближайшие 5 дней?  Доступно членам @RDVPREMIUMbot  @AK47pfl'

In [4]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
sentence = test_text

tokens = word_tokenize(sentence)

# Определение частей речи для каждого токена
tagged = pos_tag(tokens, lang='rus')

subject = None
for word, pos in tagged:
    if pos == 'S':
        subject = word
        break

print("Подлежащее:", subject)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


Подлежащее: Версия


In [5]:
#словарь новость - компании
def create_dict_mes_ids(df):
    dic = dict()
    for i in range(len(df)):
        if df.iloc[i].messageid in dic:
            dic[df.iloc[i].messageid].append(df.iloc[i].issuerid)
        else:
            dic[df.iloc[i].messageid] = [df.iloc[i].issuerid]
    return dic

mes_to_ids = create_dict_mes_ids(news_train)


In [7]:
import math
df = pd.read_excel('data/issuers_modified.xlsx')
dictionary_companies = df.set_index('issuerid').to_dict(orient='index')
dictionary_companies_by_ticker = dict()
companies = set()
for key in dictionary_companies:
    names = dictionary_companies[key]["EMITENT_FULL_NAME"]
    names_splitted = re.findall(r'"([^"]*)"', names)
    dictionary_companies[key]["EMITENT_FULL_NAME"] = names_splitted
    tickers = [dictionary_companies[key]['BGTicker']] + [dictionary_companies[key]['OtherTicker']]
    tickers = [x.split()[0] for x in tickers if not isinstance(x, float) or not math.isnan(x)]

    for ticker in tickers:
      dictionary_companies_by_ticker[ticker] = key

    for x in names_splitted:
        companies.add(x.lower())

In [80]:
#тупой способ
succes = 0
for i in range(1, len(news_train)):
    news = news_train.iloc[i] 
    company = dictionary_companies[news.issuerid]
    company_names = company['EMITENT_FULL_NAME']

    company_tickers = [company['BGTicker']] + [company['OtherTicker']]
    company_tickers = [x.split()[0] for x in company_tickers if not isinstance(x, float) or not math.isnan(x)]
    text = news.MessageText

    matches_tickers = re.findall(r'(?<=\$)\[A-Z]+|[A-Z]{4,6}', text)

    if len(matches_tickers) > 0:
      if matches_tickers[0] in company_tickers:
          succes += 1
    #print(succes, i)

print(succes / len(news_train))

KeyError: 253

In [96]:
def create_dict_mes_to_index(ms_to_copm, df):
    dic = dict()

    for ms_id in ms_to_copm:
      index = df.index[df['messageid'] == ms_id]
      #print(index[0])
      dic[ms_id] = index[0]
    return dic

mes_to_index = create_dict_mes_to_index(mes_to_ids, news_train)


In [10]:
#все тикеры компаний
df1 = df.dropna(subset=['BGTicker'])
df2 = df.dropna(subset=['OtherTicker'])

t1 = ' '.join(df1['BGTicker']).split()
t2 = ' '.join(df2['OtherTicker']).split()

all_tickers = (set(t1) | set(t2)) - set(['RX', 'LI'])
all_tickers

{'ABIO',
 'ABRD',
 'ACKO',
 'AFKS',
 'AFLT',
 'AGRO',
 'AKRN',
 'ALBK',
 'ALNU',
 'ALRS',
 'AMEZ',
 'APTK',
 'AQUA',
 'ARSA',
 'ASSB',
 'ASTR',
 'AVAN',
 'BANE',
 'BELU',
 'BISV',
 'BLNG',
 'BRZL',
 'BSPB',
 'CARM',
 'CBOM',
 'CHEP',
 'CHGZ',
 'CHKZ',
 'CHMF',
 'CHMK',
 'CIAN',
 'CNTL',
 'DASB',
 'DELI',
 'DERZP',
 'DIAS',
 'DIOD',
 'DSKY',
 'DVEC',
 'DZRD',
 'EELT',
 'ELTZ',
 'ENPG',
 'ENRU',
 'ETLN',
 'EUTR',
 'FEES',
 'FESH',
 'FIVE',
 'FIXP',
 'FLOT',
 'GAZA',
 'GAZC',
 'GAZP',
 'GAZS',
 'GAZT',
 'GCHE',
 'GECO',
 'GEMA',
 'GEMC',
 'GLPR',
 'GLTR',
 'GMKN',
 'GRNT',
 'GTLC',
 'GTRK',
 'GTSS',
 'HHRU',
 'HIMC',
 'HMSG',
 'HNFG',
 'HYDR',
 'IDVP',
 'IGST',
 'INGR',
 'IRAO',
 'IRGZ',
 'IRKT',
 'ISKJ',
 'JNOS',
 'KAZT',
 'KBSB',
 'KBTK',
 'KCHE',
 'KGKC',
 'KLSB',
 'KMAZ',
 'KMEZ',
 'KMTZ',
 'KOGK',
 'KRKN',
 'KRKO',
 'KROT',
 'KRSB',
 'KSGR',
 'KTSB',
 'KUBE',
 'KUZB',
 'KZMS',
 'KZOS',
 'LIFE',
 'LKOH',
 'LNTA',
 'LNZL',
 'LPSB',
 'LSNG',
 'LSRG',
 'LVHK',
 'MAGE',
 'MAGN',
 'MAIL',


In [97]:
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
# import pymorphy2


# morph = pymorphy2.MorphAnalyzer()

nltk.download('stopwords')
stopwords = stopwords.words('russian')
ss = SnowballStemmer(language='russian')

def extract_words(text):

    word_pattern = re.compile(r'[а-яА-я]+\b')
    words = word_pattern.findall(text)
    words = [w.lower() for w in words]
    return words


def find_company_name(text, dic):
    tickers = []
    words = extract_words(text)

    for comp_id in dic:
      names = dic[comp_id]['EMITENT_FULL_NAME']
      names = [n.lower() for n in names]
      
      
      flag =  True in [find_word_in_sentence(n, text) for n in names]

      #если нашли то добавить тикер
      if len(set(words) & set(names)) > 0 or flag:
        tickers.append(dic[comp_id]['BGTicker'])
        tickers.append(dic[comp_id]['OtherTicker'])
    tickers = [x.split()[0] for x in tickers if not isinstance(x, float) or not math.isnan(x)]

    return tickers

print(find_company_name('Это тестовое задание, Русарго СПБ северсталь', dictionary_companies))


['CHMF', 'SPBE_x000D_']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [134]:
def is_one_letter_different(word1, word2):
    if abs(len(word1) - len(word2)) > 1:
        return False

    diff_count = 0

    if len(word1) > len(word2):
        if word1[:-1] == word2:
            #print(word2, word1)
            return True
        else:
            return False

    elif len(word1) < len(word2):
        if word1 == word2[:-1]:
            #print(word2, word1)
            return True
        else:
            return False

    return word1[:-1] == word2[:-1]
    # 
    # for char1, char2 in zip(word1, word2):
    #     if char1 != char2:
    #         diff_count += 1
    #         if diff_count > 1:
    #             return False
    # if abs(len(word1) - len(word2)) == 1 and diff_count == 1:
    #     return True
    # elif len(word1) == len(word2) and diff_count == 1:
    #     return True
    # else:
    #     return False

print(is_one_letter_different("роснефти", "роснефть"))    # True
print(is_one_letter_different("cat", "cats"))   # True
print(is_one_letter_different("переговоры", "ор"))   # False
print(is_one_letter_different("яндекс", "индекс"))    # True
print(is_one_letter_different("cat", "cad"))    # True
print(is_one_letter_different("cat", "dog"))    # False

True
True
False
False
True
False


In [135]:
def find_word_in_sentence(word, sentece):
    for word2 in sentece.split():
        if len(word2) > 3:
            if is_one_letter_different(word, word2):
                #print(word, word2)
                return True

    return False

find_word_in_sentence('яндекс', 'индекс')


False

In [192]:
def split_news_on_companies(text):
    sentences = split_text_into_sentences(text)
    companies_by_index = dict()
    for i in range(len(sentences)):

        sentence = sentences[i]
        sentence = re.sub('[,;:"\>!]', '', sentence)
        #print(sentence)
        for company_ind in dictionary_companies:
            for comp_name in dictionary_companies[company_ind]['EMITENT_FULL_NAME']:
                if find_word_in_sentence(comp_name.lower(), sentence):
                    #print(i, comp_name)
                    if i in companies_by_index:
                        companies_by_index[i].add(company_ind)

                    else:
                        companies_by_index[i] = {company_ind}

        for ticker in all_tickers:

            if ticker.lower() in sentence:
                if i in companies_by_index:
                    companies_by_index[i].add(dictionary_companies_by_ticker[ticker])

                else:
                    companies_by_index[i] = {dictionary_companies_by_ticker[ticker]}
    return sentences, companies_by_index

sentences, companies_by_index = split_news_on_companies(test_text)
print(companies_by_index)
for key in companies_by_index:
    print('Sentence: ')
    print(sentences[key])
    print('Companies in news: ')
    print([dictionary_companies[ind]['EMITENT_FULL_NAME'][0] for ind in companies_by_index[key]])




{1: {57}}
Sentence: 
?#lsrg лср - операционные результаты (2019г) *продажи = -9% до 84 млрд руб *объем = -18% до 817 тыс.
Companies in news: 
['Группа ЛСР']


In [53]:
succes = 0
i = 0
error = 0
#проходим сразу по сообщениям с указанными компаниями
for mes_id in mes_to_ids:
    # if i > 1000:
    #   break

    i += len(mes_to_ids[mes_id])

    #индекс новсти в табличке
    news = news_train.iloc[mes_to_index[mes_id]]

    # 223 компании вообще нет
    if news.issuerid not in dictionary_companies:
      continue

    company = dictionary_companies[news.issuerid]
    company_names = company['EMITENT_FULL_NAME']

    company_tickers = [company['BGTicker']] + [company['OtherTicker']]
    company_tickers = [x.split()[0] for x in company_tickers if not isinstance(x, float) or not math.isnan(x)]
    text = news.MessageText


    matches_tickers = re.findall(r'(?<=\$)\[A-Z]+|[A-Z]{4,6}', text) #нашли тикеры
    founded_tickers_by_name = find_company_name(text, dictionary_companies)  # тикеры по названиям

    matches_tickers = set(matches_tickers) | set(founded_tickers_by_name)


    if len(matches_tickers) > 0:
      companies_ids = mes_to_ids[mes_id]
      if 253 in companies_ids:
        companies_ids.remove(253)
      real_tickers = [dictionary_companies[id]['BGTicker'] for id in companies_ids] + [dictionary_companies[id]['OtherTicker']  for id in companies_ids]
      real_tickers = [x.split()[0] for x in real_tickers if not isinstance(x, float) or not math.isnan(x)]

      succes += len(set(real_tickers) & set(matches_tickers))


    #можно добавить в тупую берта для большей уверенности (но плохо и долго работает)

    # else:
    #   keywords_and_scores = keybert_model.extract_keywords(text,
    #                                                      keyphrase_ngram_range=(1, 1), top_n=10)
    #   candidates = [comp[0].lower() for comp in keywords_and_scores if comp[0].lower() in companies]
    #   if len(candidates) > 0:
    #     for c in company_names:
    #       if c.lower() == candidates[0]:
    #        succes += 1
    #        break

    print(succes, i)

print(succes / len(news_train))
print(error)

#66% точности

1 1
2 2
3 3
3 5
4 6
5 7
6 8
7 9
8 10
9 11
9 13
9 14
10 15
11 17
11 18
11 19
11 21
газа газ
12 23
13 24
18 30
19 31
19 32
20 33
26 42
31 48
32 49
газа газ
32 51
32 55
34 57
34 59
35 60
35 61
35 62
36 63
37 64
38 65
39 66
40 68
газа газ
42 70
43 71
45 74
46 76
47 77
48 78
49 83
49 84
50 86
51 89
53 91
54 92
54 93
55 95
56 96
57 97
58 98
59 99
60 100
61 101
62 102
65 107
65 109
65 113
66 114
66 115
67 117
70 120
71 121
71 122
73 124
73 125
74 127
75 129
76 130
76 131
77 132
78 133
79 134
79 135
81 137
83 140
83 141
84 142
85 143
89 148
90 150
92 153
92 154
93 155
94 156
94 158
94 160
95 161
96 162
97 163
97 164
98 165
99 167
100 168
100 170
101 171
101 173
108 183
109 184
110 185
119 198
120 199
121 200
122 201
123 202
124 203
124 204
125 207
126 209
127 210
128 211
128 212
129 213
130 214
131 216
132 217
135 222
136 224
137 225
137 227
140 230
140 231
141 232
141 233
142 234
143 235
144 237
145 238
146 239
147 240
147 241
148 242
149 243
149 244
150 245
150 246
151 247
152 248
153 249
15

KeyboardInterrupt: 

In [98]:
sentiment_train

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [180]:

from tqdm.notebook import tqdm
X = []
y = []
for i in tqdm(range(len(sentiment_train))):
    text = sentiment_train.iloc[i].MessageText
    sentences, companies_by_index = split_news_on_companies(text)
    
    comp_ids_to_text = dict()
    for key in companies_by_index:
        value = companies_by_index[key]
        for company_ids in value:
            if company_ids in comp_ids_to_text:
                comp_ids_to_text[company_ids] += sentences[key]
            else:
                comp_ids_to_text[company_ids] = sentences[key]

    for key in companies_by_index:
        value = companies_by_index[key]
        for company_ids in value:
            
            if company_ids in comp_ids_to_text:
                comp_ids_to_text[company_ids] += sentences[key]
            else:
                comp_ids_to_text[company_ids] = sentences[key]
    
    
    real_comp = sentiment_train.iloc[i].issuerid
    if real_comp in comp_ids_to_text:
        X.append(comp_ids_to_text[real_comp])
        y.append(sentiment_train.iloc[i].SentimentScore)
        
        
    # print(companies_by_index)
    # for key in companies_by_index:
    #     print('Sentence: ')
    #     print(sentences[key])
    #     print('Companies in news: ')
    #     print([dictionary_companies[ind]['EMITENT_FULL_NAME'][0] for ind in companies_by_index[key]])
    # 
    # 
    #print(dictionary_companies[sentiment_train.iloc[i].issuerid]['EMITENT_FULL_NAME'][0])
    #break
    #print(i)
print(len(X))

  0%|          | 0/9289 [00:00<?, ?it/s]

8011


In [181]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stopwords = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian")
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

def prepocess_text(text):
    text = re.sub(r'[0-9,.!;:]', '', text)
    stemmed_words = [stemmer.stem(word) for word in text.split()]
    words_final = ' '.join([word for word in stemmed_words if word not in stopwords])
    return words_final
print(prepocess_text('Тестовая темка упала с крыши ой '))


тестов темк упа крыш


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [182]:
X_good = [prepocess_text(x) for x in X]

In [183]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000) #max_features 10000 
vectors = vectorizer.fit_transform(X_good)
vectors.shape

(8011, 1000)

In [191]:
import joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [184]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.2, random_state=69)

In [187]:
sv = svm.SVC()
sv.fit(X_train, y_train)

SVC()

In [189]:
from sklearn.metrics import accuracy_score, f1_score
y_pred = sv.predict(X_test)
ac = accuracy_score(y_test, y_pred)
def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2
final_score(y_test, y_pred)

0.6275092713292413

In [190]:
import joblib
joblib.dump(sv, 'svс_model_weights.pkl')

['svс_model_weights.pkl']

In [169]:
test_text = news_train.iloc[1].MessageText
test_text

'??#LSRG  ЛСР - операционные результаты (2019г) *продажи = -9% до 84 млрд руб *объем = -18% до 817 тыс. кв. м *средняя цена реализации =+15% до 112 тыс руб /кв м *введение в эксплуатацию чистой продаваемой площади = -17% до 834 тыс кв м'

In [49]:
#def return_companies_by_sentence(text)
import nltk
nltk.download('punkt')

# Splitting Text into Sentences
def split_text_into_sentences(test_text):
    test_text = ' '.join([w.lower() for w in test_text.split()])
    sentences = nltk.sent_tokenize(test_text)
    return sentences


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
sentiment_train

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [24]:
test = sentiment_train.iloc[0].MessageText

test

'⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 2022г не выплачивать'

тестов темк упа крыш


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielbelikov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [173]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X = sentiment_train[sentiment_train['SentimentScore'] > 0].MessageText
y = filtered_rows = sentiment_train[sentiment_train['SentimentScore'] > 0].SentimentScore

X_good = [prepocess_text(x) for x in X]

len(X)

9126

In [174]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=1000) #max_features 10000 
vectors = vectorizer.fit_transform(X_good)
vectors.shape

(9126, 1000)

In [175]:
dense_vectors = vectors.todense()

In [176]:
from sklearn import svm
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(dense_vectors, y, test_size=0.2, random_state=69)

In [177]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [178]:
sv = svm.SVC()
sv.fit(X_train, y_train)

SVC()

In [179]:
from sklearn.metrics import accuracy_score, f1_score
y_pred = sv.predict(X_test)
ac = accuracy_score(y_test, y_pred)
def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2
final_score(y_test, y_pred)

0.653893253235775

In [160]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [161]:
final_score(y_test, y_pred)

0.604217649095943

In [92]:
y_pred = sv.predict(X_test)
ac = accuracy_score(y_test, y_pred)

In [99]:
from sklearn.metrics import accuracy_score, f1_score

def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2

In [101]:
print(final_score(y_test, y_pred))

0.6421193070682276


In [105]:
with open('data/sentiment_texts_demoji.pickle',  'rb') as f:
    emoji = pickle.load(f)
    
emoji

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,:warning::Russia:#SELG #дивиденд сд Селигдар:...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются:fire:Акции и инвестиции ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются:fire:Акции и инвестиции ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды :money_bag: 7% — возможная ди...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,:Russia:#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВК...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля :small_b...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,""":collision::Russia:#PLZL #листинг #торги """"П...",False


In [106]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X_e = emoji.MessageText
y_e = emoji.SentimentScore

X_good_e = [prepocess_text(x) for x in X_e]

In [107]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=1000) #max_features 10000 65
vectors_e = vectorizer.fit_transform(X_good_e)
vectors_e.shape

(9289, 1000)

In [108]:
from sklearn import svm
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(vectors_e, y_e, test_size=0.2, random_state=69)
sv = svm.SVC()
sv.fit(X_train, y_train)

SVC()

In [110]:
y_pred = sv.predict(X_test)
ac = accuracy_score(y_test, y_pred)

In [111]:
ac

0.6480086114101185

In [112]:
from sklearn.metrics import accuracy_score, f1_score
y_pred = sv.predict(X_test)
ac = accuracy_score(y_test, y_pred)
def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2
final_score(y_test, y_pred)

0.6379098064307729